In [1]:
from keras.models import model_from_json
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
file = open("Emotion.json", 'r')
model_json = file.read()

loaded_model = model_from_json(model_json)

# load weights
loaded_model.load_weights("Emotion_model_weights.h5")

In [3]:
import cv2 as cv
video = cv.VideoCapture(0) # 0 for in-built webcam and 1 for external web-cam/cameras
labels = {"persons_name" : 1}
with open("labels.pickle", "rb") as f:
    og_labels = pickle.load(f)
    labels = { v:k for k,v in og_labels.items()}
print(labels)

{0: 'happy', 1: 'disgust', 2: 'anger', 3: 'contempt', 4: 'sadness', 5: 'surprise', 6: 'fear'}


In [4]:
loaded_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [5]:
face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
pred = np.zeros((7,1))
pred = [[0],[1],[2],[3],[4],[5],[6]]
while True:
    ret, frame = video.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.5,minNeighbors = 5 )
    for (x,y,w,h) in faces:
        #print(x,y,w,h)
        roi_gray = gray[y:y+h,x:x+w]
        color = (255,0,255)
        stroke = 2
        cv.rectangle(frame,(x,y),(x+w,y+h), color, stroke)
        input_shape = (1,128,128,1)
        #print(roi_gray)
        #roi_gray = cv.resize(100,100)
        img_item = "my_image.png"
        cv.imwrite(img_item,roi_gray)
        img = cv.imread(img_item,0)
        img_2 = cv.resize(img,(128,128))
        my_array = np.ndarray(shape = input_shape)
        my_array = np.asarray(img_2)
        #print(my_array.shape)
        my_array = my_array.reshape(input_shape)
        #print(type(roi_gray.shape))
        Y_pred = loaded_model.predict(my_array)
        predictions1 = np.dot(Y_pred,pred)
        predictions1 = np.array(predictions1, dtype = int)
        predict1 = np.ndarray.flatten(predictions1)
        x = predict1.item()
        #print(x)
        #print(predict1.item())
        #print(labels[x])
        font  = cv.FONT_HERSHEY_SIMPLEX
        name = labels[x]
        color = (55,25,255)
        stroke = 2
        cv.putText(frame, name, (x,y), font,1 , color, stroke, cv.LINE_AA)

    cv.imshow("Frame", frame)
    key = cv.waitKey(1)
    
    if key == ord("q"):
        break
cv.destroyAllWindows()
video.release()